##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Tutorial de componentes de função Python do TFX


Observação: recomendamos executar este tutorial em um notebook Colab, sem necessidade de configuração! Basta clicar em “Executar no Google Colab”.

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td>     <a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/python_function_component"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/tfx/python_function_component.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
<td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/tfx/python_function_component.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
<td>     <a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tfx/tutorials/tfx/python_function_component.ipynb"><img width="32px" src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table></div>

Este notebook contém exemplos sobre como criar e executar componentes de função Python dentro do TFX InteractiveContext e num pipeline TFX orquestrado localmente.

Para obter mais contexto e informações, consulte a página [Componentes de função Python personalizados](https://www.tensorflow.org/tfx/guide/custom_function_component) no site de documentação do TFX.

## Configuração

Primeiro instalaremos o TFX e importaremos os módulos necessários. TFX requer Python 3.

### Verifique a versão do sistema Python


In [ ]:
import sys
sys.version

### Atualize o Pip

Para evitar a atualização do Pip num sistema ao executar localmente, garanta que estamos executando no Colab. Os sistemas locais podem, claro, ser atualizados separadamente.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### Instale o TFX

**Observação: no Google Colab, devido a atualizações de pacotes, na primeira vez que você executar esta célula você deverá reiniciar o runtime (Runtime &gt; Restart runtime...).**

In [ ]:
!pip install tfx

### Desinstale o shapely

TODO(b/263441833) Esta é uma solução temporária para evitar um ImportError. Em última análise, isto deverá ser resolvido com suporte a uma versão mais recente do Bigquery, em vez de desinstalar outras dependências extras.

In [ ]:
!pip uninstall shapely -y

## Você reiniciou o runtime?

Se você estiver usando o Google Colab, na primeira vez que executar a célula acima, você deve reiniciar o runtime ("Runtime &gt; Restart runtime ..."). Isso é necessário devido à maneira como o Colab carrega os pacotes.

### Importe os pacotes

Importamos o TFX e verificamos sua versão.

In [ ]:
# Check version
from tfx import v1 as tfx
tfx.__version__

## Componentes de função Python personalizados

Nesta seção, criaremos componentes a partir de funções Python. Não resolveremos nenhum problema real de ML - essas funções simples são usadas apenas para ilustrar o processo de desenvolvimento do componente de função Python.

Consulte o [Guia de componentes baseados em funções Python](https://www.tensorflow.org/tfx/guide/custom_function_component) para mais informações.

### Crie componentes em Python personalizados

Começamos escrevendo uma função que gera alguns dados fictícios. Isso é gravado em seu próprio arquivo de módulo Python.

In [ ]:
%%writefile my_generator.py

import os
import tensorflow as tf  # Used for writing files.

from tfx import v1 as tfx

# Non-public APIs, just for showcase.
from tfx.types.experimental.simple_artifacts import Dataset

@tfx.dsl.components.component
def MyGenerator(data: tfx.dsl.components.OutputArtifact[Dataset]):
  """Create a file with dummy data in the output artifact."""
  with tf.io.gfile.GFile(os.path.join(data.uri, 'data_file.txt'), 'w') as f:
    f.write('Dummy data')

  # Set metadata and ensure that it gets passed to downstream components.
  data.set_string_custom_property('my_custom_field', 'my_custom_value')

Em seguida, escrevemos um segundo componente que usa os dados fictícios produzidos. Iremos apenas calcular o hash dos dados e devolvê-los.

In [ ]:
%%writefile my_consumer.py

import hashlib
import os
import tensorflow as tf

from tfx import v1 as tfx

# Non-public APIs, just for showcase.
from tfx.types.experimental.simple_artifacts import Dataset
from tfx.types.standard_artifacts import String

@tfx.dsl.components.component
def MyConsumer(data: tfx.dsl.components.InputArtifact[Dataset],
               hash: tfx.dsl.components.OutputArtifact[String],
               algorithm: tfx.dsl.components.Parameter[str] = 'sha256'):
  """Reads the contents of data and calculate."""
  with tf.io.gfile.GFile(
      os.path.join(data.uri, 'data_file.txt'), 'r') as f:
    contents = f.read()
  h = hashlib.new(algorithm)
  h.update(tf.compat.as_bytes(contents))
  hash.value = h.hexdigest()

  # Read a custom property from the input artifact and set to the output.
  custom_value = data.get_string_custom_property('my_custom_field')
  hash.set_string_custom_property('input_custom_field', custom_value)

### Execute no notebook com o InteractiveContext

Agora, demonstraremos o uso de nossos novos componentes no TFX InteractiveContext.

Para obter mais informações sobre o que você pode fazer com o notebook TFX InteractiveContext, veja o [Tutorial do componente TFX Keras](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras) no notebook.

In [ ]:
from my_generator import MyGenerator
from my_consumer import MyConsumer

#### Construa o InteractiveContext

In [ ]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
context = InteractiveContext()

#### Execute seu componente interativamente com `context.run()`

Em seguida, executamos nossos componentes interativamente no notebook com `context.run()`. Nosso componente consumidor utiliza as saídas do componente gerador.

In [ ]:
generator = MyGenerator()
context.run(generator)

In [ ]:
consumer = MyConsumer(
    data=generator.outputs['data'],
    algorithm='md5')
context.run(consumer)

Após a execução, podemos inspecionar o conteúdo "hash" do artefato de saída do componente consumidor no disco.

In [ ]:
!tail -v {consumer.outputs['hash'].get()[0].uri}

É isso, e agora você escreveu e executou seus próprios componentes personalizados!

### Escreva uma definição de pipeline

A seguir, criaremos um pipeline usando esses mesmos componentes. Embora o uso do `InteractiveContext` em um notebook funcione bem para experimentação, a definição de um pipeline permite implantar seu pipeline em executores locais ou remotos para uso em produção.

Aqui, demonstraremos o uso do LocalDagRunner rodando localmente em sua máquina. Para execução em produção, os executores Airflow ou Kubeflow podem ser mais adequados.

#### Construa um pipeline

In [ ]:
import os
import tempfile
from tfx import v1 as tfx

# Select a persistent TFX root directory to store your output artifacts.
# For demonstration purposes only, we use a temporary directory.
PIPELINE_ROOT = tempfile.mkdtemp()
# Select a pipeline name so that multiple runs of the same logical pipeline
# can be grouped.
PIPELINE_NAME = "function-based-pipeline"
# We use a ML Metadata configuration that uses a local SQLite database in
# the pipeline root directory. Other backends for ML Metadata are available
# for production usage.
METADATA_CONNECTION_CONFIG = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    os.path.join(PIPELINE_ROOT, 'metadata.sqlite'))

def function_based_pipeline():
  # Here, we construct our generator and consumer components in the same way.
  generator = MyGenerator()
  consumer = MyConsumer(
      data=generator.outputs['data'],
      algorithm='md5')

  return tfx.dsl.Pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      components=[generator, consumer],
      metadata_connection_config=METADATA_CONNECTION_CONFIG)

my_pipeline = function_based_pipeline()

#### Execute seu pipeline com o `LocalDagRunner`

In [ ]:
tfx.orchestration.LocalDagRunner().run(my_pipeline)

Podemos inspecionar os artefatos de saída gerados pela execução deste pipeline.

In [ ]:
!find {PIPELINE_ROOT}

Agora você escreveu seus próprios componentes personalizados e orquestrou sua execução no LocalDagRunner! Para os próximos passos, confira os tutoriais e guias adicionais no [site do TFX](https://www.tensorflow.org/tfx).